In [9]:
import os
import csv

# File reader function to read files from data source
def file_reader(dir_path, file_path):
    # Loop responsible for opening files
    data_set = []
    for file in os.listdir(dir_path):
        data = open(f'{file_path}/{file}', encoding='utf-8')
        csv_data = list(csv.reader(data))
        # Below loop applies key to every csv file cell. That data for both datasets would be homogenious
        headers = csv_data[0]
        for data_row in csv_data[1:]:
            my_dict = {}
            for i, cell in enumerate(data_row):
                my_dict[headers[i]] = cell
            data_set.append(my_dict)
    data.close()
    return data_set


In [10]:
dir_path_clicks = f'{os.getcwd()}/data/clicks'
file_path_clicks = 'data/clicks'

dir_path_users = f'{os.getcwd()}/data/users'
file_path_users = 'data/users'

data_set_clicks = file_reader(dir_path_clicks, file_path_clicks)
data_set_users = file_reader(dir_path_users, file_path_users)

print(data_set_clicks)
print(data_set_users)

[{'date': '2017-12-20', 'screen': 'feed', 'user_id': '3', 'click_target': 'ad'}, {'date': '2017-12-19', 'screen': 'catalog', 'user_id': '3', 'click_target': 'ad'}, {'date': '2017-12-21', 'screen': 'feed', 'user_id': '3', 'click_target': 'profile'}, {'date': '2017-12-10', 'screen': 'catalog', 'user_id': '7', 'click_target': 'ad'}, {'date': '2017-12-12', 'screen': 'feed', 'user_id': '3', 'click_target': 'ad'}, {'date': '2017-12-17', 'screen': 'feed', 'user_id': '3', 'click_target': 'item'}, {'date': '2017-12-13', 'screen': 'feed', 'user_id': '7', 'click_target': 'item'}, {'date': '2017-12-10', 'screen': 'feed', 'user_id': '7', 'click_target': 'profile'}, {'date': '2017-12-20', 'screen': 'feed', 'user_id': '7', 'click_target': 'profile'}, {'date': '2017-12-17', 'screen': 'catalog', 'user_id': '7', 'click_target': 'ad'}, {'date': '2017-12-12', 'screen': 'feed', 'user_id': '7', 'click_target': 'ad'}, {'date': '2017-12-13', 'screen': 'blog', 'user_id': '7', 'click_target': 'ad'}, {'date': '2

In [11]:
# Mapper function responsible of generating key-alue dictionary pairs and returning them
def mapper_task_1(data):
    results = []
    for click in data:
        key = click['date']
        value = click
        results.append(({'key': key, 'value': value}))
    return results

# Reducer funtion responsible for count, how many each day there were clicks
def reducer_task_1(key, values):
    result = []
    result.append({'date': key, 'count': len(values)})
    # Returning all key-value pairs
    return result


In [13]:
# TESTING mapper_task_1 (reducer will be tested with map_reduce implementation, when data will be grouped)
test_m_1 = mapper_task_1(data_set_clicks)
print(test_m_1)

[{'key': '2017-12-20', 'value': {'date': '2017-12-20', 'screen': 'feed', 'user_id': '3', 'click_target': 'ad'}}, {'key': '2017-12-19', 'value': {'date': '2017-12-19', 'screen': 'catalog', 'user_id': '3', 'click_target': 'ad'}}, {'key': '2017-12-21', 'value': {'date': '2017-12-21', 'screen': 'feed', 'user_id': '3', 'click_target': 'profile'}}, {'key': '2017-12-10', 'value': {'date': '2017-12-10', 'screen': 'catalog', 'user_id': '7', 'click_target': 'ad'}}, {'key': '2017-12-12', 'value': {'date': '2017-12-12', 'screen': 'feed', 'user_id': '3', 'click_target': 'ad'}}, {'key': '2017-12-17', 'value': {'date': '2017-12-17', 'screen': 'feed', 'user_id': '3', 'click_target': 'item'}}, {'key': '2017-12-13', 'value': {'date': '2017-12-13', 'screen': 'feed', 'user_id': '7', 'click_target': 'item'}}, {'key': '2017-12-10', 'value': {'date': '2017-12-10', 'screen': 'feed', 'user_id': '7', 'click_target': 'profile'}}, {'key': '2017-12-20', 'value': {'date': '2017-12-20', 'screen': 'feed', 'user_id': 

In [16]:
# CUSTOM mapReduce IMPLEMENTATION
# As number of datasets differs in both tasks, mapReduce takes list of datasets and list of mappers (if only one should be passed, it should be in a list as well)
def map_reduce(datasets, mappers, reducer):
    # Apply the mapper to the datasets
    intermediate_data = []
    for i, dataset in enumerate(datasets):
        mapper = mappers[i]
        intermediate_data += mapper(dataset)
    # Group the intermediate data by key
    grouped_data = {}
    for item in intermediate_data:
        if item['key'] in grouped_data:
            grouped_data[item['key']].append(item['value'])
        else:
            grouped_data[item['key']] = [item['value']]
    # Apply the reducer to each group of values
    final_data = []
    for key, values in grouped_data.items():
        final_data.extend(reducer(key, values))
    # Return the final output
    return final_data

In [19]:
# TESTING map_reduce + reducer for FIRST task
final_data_1 = map_reduce(datasets=[data_set_clicks], mappers=[mapper_task_1], reducer=reducer_task_1)
for data in final_data_1:
    print(data)

{'date': '2017-12-20', 'count': 13}
{'date': '2017-12-19', 'count': 8}
{'date': '2017-12-21', 'count': 7}
{'date': '2017-12-10', 'count': 4}
{'date': '2017-12-12', 'count': 12}
{'date': '2017-12-17', 'count': 12}
{'date': '2017-12-13', 'count': 9}
{'date': '2017-12-14', 'count': 10}
{'date': '2017-12-16', 'count': 10}
{'date': '2017-12-11', 'count': 4}
{'date': '2017-12-15', 'count': 6}
{'date': '2017-12-18', 'count': 5}


In [63]:
# TWO MAP FUNCTIONS FOR SECOND TASK
# Same return as mapper function in first task, just additional implementation of table key value pair
# 'table' : 'users' <-- for users dataset
# 'table' : 'clicks' <-- for clicks dataset
def mapper_task_2_users(users):
    results = []
    for user in users:
        if user['country'] == 'LT':
            key = user['id']
            value = user.copy()
            # indicator that this is a users table
            value['table'] = 'users'
            results.append({'key': key, 'value': value})
    return results

def mapper_task_2_clicks(clicks):
    results = []
    for click in clicks:
        key = click['user_id']
        value = click
        # indicator that this is a users table
        value['table'] = 'clicks'
        results.append(({'key': key, 'value': value}))
    return results

# Reducer accepts mapped data (LT user + his clicks) + unmatched clicks, which are later filtered out
def reducer_task_2(key, values):
    # Values -> LT user : his click | LT user : no clicks | clicks without LT user (first mapper filters out users that are not from LT)
    # user -> Filters out from values onlt user data (needed to filter out clicks)
    user = next((value for value in values if value['table'] == 'users'), None)
    filtered_clicks = []
    for click in values:
        # From grouped dataset takes only data that are clicks
        # And merges user data with click data (like join) (user data + click data) --> single line
        # After that it appends all merged clicks to filtered list (user data + click data) --> multiple lines
        if click['table'] == 'clicks':
            merged_click = click.copy()
            if user is not None:
                merged_click.update(user)
            if 'country' in merged_click and merged_click['country'] is not None:
                filtered_clicks.append(merged_click)
    return filtered_clicks



In [64]:
# TESTING mapper_task_2
# TESTING mapper_task_1 (reducer will be tested with map_reduce implementation, when data will be grouped)
test_m_2_u = mapper_task_2_users(data_set_users)
print(test_m_2_u)

test_m_2_c = mapper_task_2_clicks(data_set_clicks)
print(test_m_2_c)


[{'key': '1', 'value': {'id': '1', 'country': 'LT', 'table': 'users'}}, {'key': '3', 'value': {'id': '3', 'country': 'LT', 'table': 'users'}}, {'key': '100', 'value': {'id': '100', 'country': 'LT', 'table': 'users'}}, {'key': '300', 'value': {'id': '300', 'country': 'LT', 'table': 'users'}}, {'key': '4', 'value': {'id': '4', 'city': 'Vilnius', 'country': 'LT', 'table': 'users'}}, {'key': '102', 'value': {'id': '102', 'city': 'Kaunas', 'country': 'LT', 'table': 'users'}}, {'key': '103', 'value': {'id': '103', 'city': 'Vilnius', 'country': 'LT', 'table': 'users'}}]
[{'key': '3', 'value': {'date': '2017-12-20', 'screen': 'feed', 'user_id': '3', 'click_target': 'ad', 'table': 'clicks'}}, {'key': '3', 'value': {'date': '2017-12-19', 'screen': 'catalog', 'user_id': '3', 'click_target': 'ad', 'table': 'clicks'}}, {'key': '3', 'value': {'date': '2017-12-21', 'screen': 'feed', 'user_id': '3', 'click_target': 'profile', 'table': 'clicks'}}, {'key': '7', 'value': {'date': '2017-12-10', 'screen': 

In [65]:
final_data_2 = map_reduce(datasets=[data_set_users, data_set_clicks], mappers=[mapper_task_2_users, mapper_task_2_clicks], reducer=reducer_task_2)
for data in final_data_2:
    print(data)

{'date': '2017-12-11', 'user_id': '1', 'click_target': 'ad', 'table': 'users', 'id': '1', 'country': 'LT'}
{'date': '2017-12-20', 'user_id': '1', 'click_target': 'ad', 'table': 'users', 'id': '1', 'country': 'LT'}
{'date': '2017-12-20', 'user_id': '1', 'click_target': 'item', 'table': 'users', 'id': '1', 'country': 'LT'}
{'date': '2017-12-21', 'user_id': '1', 'click_target': 'ad', 'table': 'users', 'id': '1', 'country': 'LT'}
{'date': '2017-12-13', 'user_id': '1', 'click_target': 'item', 'table': 'users', 'id': '1', 'country': 'LT'}
{'date': '2017-12-12', 'user_id': '1', 'click_target': 'ad', 'table': 'users', 'id': '1', 'country': 'LT'}
{'date': '2017-12-21', 'user_id': '1', 'click_target': 'profile', 'table': 'users', 'id': '1', 'country': 'LT'}
{'date': '2017-12-14', 'user_id': '1', 'click_target': 'ad', 'table': 'users', 'id': '1', 'country': 'LT'}
{'date': '2017-12-17', 'user_id': '1', 'click_target': 'item', 'table': 'users', 'id': '1', 'country': 'LT'}
{'date': '2017-12-12', 'us